In [1]:
%%capture
!pip install pandas
!pip install beautifulsoup4

In [34]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# import pdb

def scrape_page(url):
    html_string = requests.get(url).text
    soup = BeautifulSoup(html_string, 'html.parser')

    # Find all tbody elements in the HTML
    tbodies = soup.find_all('tbody')

    # If no tbody elements are found, return an empty DataFrame
    if not tbodies:
        return pd.DataFrame()

    # We'll store each table's data in this list
    all_data = []

    for tbody in tbodies:
        # Now, iterate over each row in tbody
        for row in tbody.find_all('tr'):
            name_cell = row.find('th', scope='row')
            number_cell = row.find('td')

            if name_cell and number_cell:
                name = name_cell.get_text(strip=True)
                number = number_cell.get_text(strip=True)
                all_data.append([name, number])

    return pd.DataFrame(all_data, columns=['Name', 'Number'])




In [35]:
def filter_df(df):
  return df[df['Number'].str.isnumeric()]

In [36]:
# scrape_page("https://ind.nl/en/public-register-recognised-sponsors/public-register-regular-labour-and-highly-skilled-migrants")

,Name,Number
0,PTW Benelux B.V.,24395334
1,@EasePay B.V.,83892869
2,@Fentures B.V.,82701695
3,01-10 Architecten B.V.,24257403
4,050media,58806415
...,...,...
10401,ZV Control & Finance B.V.,81608195
10402,ZVC Netherlands B.V.,72895977
10403,Zycus Infotech Netherlands B.V.,71493166
10404,ZyLAB Technologies B.V.,34284277


In [37]:
urls=["https://ind.nl/en/public-register-regular-labour-and-highly-skilled-migrants", \
      "https://ind.nl/en/public-register-au-pair-and-exchange",\
      "https://ind.nl/en/public-register-au-pair-and-exchange", \
      "https://ind.nl/en/public-register-scientific-researcher-directive-eu-2016801"]
dfs=[]
for url in urls:
  df = scrape_page(url)
  df = filter_df(df)
  dfs.append(df)



In [38]:
result = {"sponsors": []}
for df in dfs:
  result["sponsors"]+=df.Name.values.tolist()

In [39]:
import json
# Convert the dictionary to a JSON string
json_data = json.dumps(result)

# Save the JSON string to a file
with open("km_company.json", "w") as file:
    file.write(json_data)